# Linear Models

In [6]:
import pandas as pd
import numpy as np

In [20]:
#this chunk written by Ana Boeriu
class Functions:
    """
     Conatins all cleaning function for this data
    """
    
#     def __init__(self,df):

    def getNAs(self, df):
        colNames = []
        percentNA = []
        for i in df.columns:
            colNames.append(i)
            numNA = df[i].isna().sum()
            percent = (numNA/len(df))*100
            percentNA.append(percent)

        colNames = pd.DataFrame(colNames)
        colNames = colNames.rename(columns={0: "label"})
        percentNA = pd.DataFrame(percentNA)
        percentNA = percentNA.rename(columns={0: "numNA"})
        self.data = pd.concat([colNames,percentNA], axis = 1).sort_values(by=['numNA'], ascending = False)

        return self.data
        
    def drop_columns(self, df, threshold_percent):
        '''drop columns by a threshold (percentage of na)'''
        global data ### double check...
        data = self.getNAs(df)
        column_names = data[data.numNA >= threshold_percent].label
        self.clean_data = df.drop(column_names, axis = 1)

        return self.clean_data
    
    def entry_to_lowercase(self, df):
        for i in df.columns:
            if (df[i].dtype == "O"):
                df[i] = df[i].str.lower()
        return df
    
    def id_data_types(self, df):
        names = pd.DataFrame()
        category = pd.DataFrame()
        data_type = pd.DataFrame()
        numUni = pd.DataFrame()

        for i in df.columns:

            names = names.append({"variable": i},ignore_index = True)
            category = category.append({"category": list(df[i].unique())}, ignore_index = True)
            numUni = numUni.append({"numUnique": len(df[i].unique())}, ignore_index = True)
            data_type = data_type.append({"data_type": df[i].dtype}, ignore_index = True)

        view_data = pd.concat([names, category, numUni,data_type],axis =1)

        return (view_data)
    
    def replace_na_with_NaN(self, df):
        
        """
         Input: Full dataset
         Output: Full dataset where object strings have underscores and spaces removed
         EX:'No_School-> NoSchool, no school -> noschool'
        """
        
        for i in df.columns:
            df[i] = df[i].replace(np.nan, 'na')
        return df
    
    # removes underscores and spaces from instances, 
    def remove_underscores_spaces(self, df):
        
        """
         Input: Full dataset
         Output: Full dataset where object strings have underscores and spaces removed
         EX:'No_School-> NoSchool, no school -> noschool'
        """
        
        for i in df.columns:
            if (df[i].dtype == "O"):
                df[i] = df[i].str.replace('_', "")
                df[i] = df[i].str.replace(' ', "")
        return df
    
    
    
    def convert_to_categorical(self, df, variables):
        """
         Input: dataframe, column names of variables to be transformed
         Output: dataframe with dtypes of indicated variables changed
         
         #NOTE: USE this function AFTER preprocessing functions above
        """
    
    # convert these to categories
    #df[v1] = df[v1].astype(‘category’)
    
        for i in variables:
            df[i] = df[i].astype('category')
        return df
    
    
    def impute_data(self, Xtrain, Xtest):
        """
        Input: XTraining set, XTesting Set
        Output: The inputs with NA numerical variabes imputed by column median, and categorical
        NA varaibles converted into a seperate category "Na" 
        """
        
       
   
        num_train = Xtrain.select_dtypes(include=['float64', 'int64'])
        num_test = Xtest.select_dtypes(include=['float64', 'int64'])
        
        cat_train = Xtrain.select_dtypes(include=['object', 'category'])
        cat_test = Xtest.select_dtypes(include=['object', 'category'])
      

    # imputes NA numerics with median
        for i in num_train.columns:
            Xtrain[i] = num_train.fillna(np.nanmedian(Xtrain[i]))
        
        for i in num_test.columns:
            Xtest[i] = num_test.fillna(np.nanmedian(num_test[i]))
        
    # categorical NA to "Na" level
        for i in cat_train.columns:
            Xtrain[i] = cat_train.replace(np.nan, "Na")
        
        for i in cat_test.columns:
            Xtest[i] = cat_test.replace(np.nan, "Na")

   
        return Xtrain, Xtest

    def drop_response_rows_with_NAs(self,df,y_variable):
        """
        Input: dataframe, response variable column 
        Output: dataframe rows with missing response varaibles
        are dropped entirely (we cannot use these to train or test)
        But these dropped rows are saved to be used later
        
        """
    
    ## need to delete PPI_threshold since that is like our y2 which we won’t use
        df = df.drop("PPI_Threshold", axis=1)
        condition = df[y_variable]
        rows_to_delete = df[np.isnan(condition)].index
    
    #save all rows with y=NA. Will use this data to predict the y later.
        prediction_dataset = df.iloc[rows_to_delete,:]
    
    #create new data
        new_data = df.drop(labels=rows_to_delete, axis=0)
    
        return new_data, prediction_dataset

In [21]:
clean = pd.read_csv("../data/RHoMIS_Full_Data.csv", engine = "python")
clean.head()

,ID_PROJ,ID_COUNTRY,YEAR,ITERATION,SURVEY_ID,ID_HH,GPS_LAT,GPS_LON,GPS_ALT,start_time_user,...,Additional_respondents,quality_rapport,quality_reliability,end_time_user,endtime_auto,endtime_calculated,X_submission_time,X_tags,X_version,X_duration
0,CW2,BF,2015,1,BF_CW2_2015,BF_2015_CW2_182_1,NaN,NaN,NaN,na,...,na,na,na,na,na,na,na,na,na,na
1,CW2,BF,2015,1,BF_CW2_2015,BF_2015_CW2_7_1,NaN,NaN,NaN,na,...,na,na,na,na,na,na,na,na,na,na
2,CW2,BF,2015,1,BF_CW2_2015,BF_2015_CW2_25_1,NaN,NaN,NaN,na,...,na,na,na,na,na,na,na,na,na,na
3,CW2,BF,2015,1,BF_CW2_2015,BF_2015_CW2_39_1,NaN,NaN,NaN,na,...,na,na,na,na,na,na,na,na,na,na
4,CW2,BF,2015,1,BF_CW2_2015,BF_2015_CW2_24_1,NaN,NaN,NaN,na,...,na,na,na,na,na,na,na,na,na,na


In [22]:
from Functions import Cleaning_Functions
fun = Cleaning_Functions()

In [27]:
market_data = fun.delete_id_columns(clean) #1
print("1")
#market_data, pred_market = fun.drop_response_rows_with_NAs(market_data, "Market_Orientation") #2
market_data = fun.replace_NAN_with_na(market_data) #3
print("3")
market_data = fun.entry_to_lowercase(market_data) #4
print("4")
market_data = fun.remove_underscores_spaces(market_data) #5
print("5")
market_data = fun.convert_to_categorical(market_data) #6
print("6")
market_data = fun.impute_data(market_data)
print("7")

1
3
4
5


KeyboardInterrupt: 

In [37]:
clean.loc[:, "Market_Orientation"]

KeyError: 'Market_Orientation'

In [29]:
from sklearn.model_selection import train_test_split

X_m = market_data.loc[:, market_data.columns != 'Market_Orientation']
y_m = market_data.loc[:, "Market_Orientation"]
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X,y,test_size = .3)

X_train_m, X_test_m= fun.impute_data(X_train_m, X_test_m)

X_train_m

KeyError: 'Market_Orientation'